In [4]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://pp

In [5]:
# Connecting to PostgreSQL
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-10-24 05:09:35--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.31MB/s    in 0.7s    

2021-10-24 05:09:37 (1.31 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [6]:
# Build a spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [7]:
# Extract Video Game reviews from S3 bucket
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
video_games_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True )

# Show DataFrame
video_games_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [8]:
# Count the number of rows in video_games_df
record_count = video_games_df.count()

print(f'There are {record_count} records in this dataframe')


There are 1785997 records in this dataframe


In [9]:
# Transform video_games_df into the review_id_table schema in resources
from pyspark.sql.types import DateType

# Create a list of the columns we want
review_id_columns = ['review_id' , 'customer_id' , 'product_id' , 'product_parent' , 'review_date']

# Subset video_games_df using the list
review_id_df = video_games_df[review_id_columns]

review_id_df.show()

# Check that the dtypes match what is requested in the schema
# review_date is string , that needs to change

# Convert review_date to date type
review_id_df = review_id_df.withColumn('review_date' , review_id_df['review_date'].cast(DateType()))

# confirm type conversion
review_id_df.dtypes





+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RTIS3L2M1F5SM|   12039526|B001CXYMFS|     737716809| 2015-08-31|
| R1ZV7R40OLHKD|    9636577|B00M920ND6|     569686175| 2015-08-31|
|R3BH071QLH8QMC|    2331478|B0029CSOD2|      98937668| 2015-08-31|
|R127K9NTSXA2YH|   52495923|B00GOOSV98|      23143350| 2015-08-31|
|R32ZWUXDJPW27Q|   14533949|B00Y074JOM|     821342511| 2015-08-31|
|R3AQQ4YUKJWBA6|    2377552|B002UBI6W6|     328764615| 2015-08-31|
|R2F0POU5K6F73F|   17521011|B008XHCLFO|      24234603| 2015-08-31|
|R3VNR804HYSMR6|   19676307|B00BRA9R6A|     682267517| 2015-08-31|
| R3GZTM72WA2QH|     224068|B009EPWJLA|     435241890| 2015-08-31|
| RNQOY62705W1K|   48467989|B0000AV7GB|     256572651| 2015-08-31|
|R1VTIA3JTYBY02|     106569|B00008KTNN|     384411423| 2015-08-31|
|R29DOU8791QZL8|   48269642|B000A3IA0Y|     472622859| 2015-08

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [16]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.c0iyntsfllgk.us-west-1.rds.amazonaws.com:5432/HW22"
config = {"user":"root", 
          "password": "Postgres1", 
          "driver":"org.postgresql.Driver"}


In [ ]:
# writing to review_id_table in PGadmin
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [18]:
# Creating df to match products schema

# Creating a list of the columns needed
productList = ['product_id' , 'product_title']

# Creating a subset from video_games_df
products_df = video_games_df[[productList]]

products_df.show()


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001CXYMFS|Thrustmaster T-Fl...|
|B00M920ND6|Tonsee 6 buttons ...|
|B0029CSOD2|Hidden Mysteries:...|
|B00GOOSV98|GelTabz Performan...|
|B00Y074JOM|Zero Suit Samus a...|
|B002UBI6W6|Psyclone Recharge...|
|B008XHCLFO|Protection for yo...|
|B00BRA9R6A|   Nerf 3DS XL Armor|
|B009EPWJLA|One Piece: Pirate...|
|B0000AV7GB|Playstation 2 Dan...|
|B00008KTNN|Metal Arms: Glitc...|
|B000A3IA0Y|72 Pin Connector ...|
|B0053BQN34|uDraw Gametablet ...|
|B002I0HIMI|NBA 2K12(Covers M...|
|B0081EH12M|New Trigger Grips...|
|B005FMLZQQ|Xbox 360 Media Re...|
|B002BSA388|Super Mario Galaxy 2|
|B00BUSLSAC|Nintendo 3DS XL -...|
|B00KWF38AW|Captain Toad:  Tr...|
|B00BRQJYA8|Lego Batman 2: DC...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Writing products_df to the products schema
review_id_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [26]:
# creating a df to match the customers schema
from pyspark.sql.functions import lit

# Creating subset of video_games_df
customers_df = video_games_df[customersList]


# creating the customer count column
customers_df = customers_df.withColumn('customer_count' ,lit(record_count) )
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   12039526|       1785997|
|    9636577|       1785997|
|    2331478|       1785997|
|   52495923|       1785997|
|   14533949|       1785997|
|    2377552|       1785997|
|   17521011|       1785997|
|   19676307|       1785997|
|     224068|       1785997|
|   48467989|       1785997|
|     106569|       1785997|
|   48269642|       1785997|
|   52738710|       1785997|
|   10556786|       1785997|
|    2963837|       1785997|
|   23092109|       1785997|
|   23091728|       1785997|
|   10712640|       1785997|
|   17455376|       1785997|
|   14754850|       1785997|
+-----------+--------------+
only showing top 20 rows



[('customer_id', 'int'), ('customer_count', 'int')]

In [ ]:
# writing customers_df to the customers schema
review_id_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [29]:
# creating a df to match the vine schema

# listing the needed columns
vineColumns = ['review_id' , 'star_rating' , 'helpful_votes' , 'total_votes' , 'vine']

# creating a subset of video_games_df
vine_df = video_games_df[vineColumns]
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RTIS3L2M1F5SM|          5|            0|          0|   N|
| R1ZV7R40OLHKD|          5|            0|          0|   N|
|R3BH071QLH8QMC|          1|            0|          1|   N|
|R127K9NTSXA2YH|          3|            0|          0|   N|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|
|R2F0POU5K6F73F|          5|            0|          0|   N|
|R3VNR804HYSMR6|          5|            0|          0|   N|
| R3GZTM72WA2QH|          5|            0|          0|   N|
| RNQOY62705W1K|          4|            0|          0|   N|
|R1VTIA3JTYBY02|          5|            0|          0|   N|
|R29DOU8791QZL8|          1|            0|          0|   N|
|R15DUT1VIJ9RJZ|          2|            0|          0|   N|
|R3IMF2MQ3OU9ZM|          4|            

In [ ]:
# writing to the vine_table using vine_df
review_id_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)